In [1]:
import pyautogui as pag
import numpy as np
import mss, cv2, win32gui, win32ui, win32con, pdb, time, mouse
import matplotlib.pyplot as plt
import copy
from ohouse import *
import multiprocessing as mp
from importlib import reload
import pdb

In [2]:
challenge_mode_pos = [220, 719]
practice_mode_pos = [220, 779]
tile_xpos = np.arange(4)*100 + 50
tile_ypos = np.arange(4, dtype=int)*100 + 550

In [14]:
mode = 'challenge'
# mode = 'practice'
start_pos = challenge_mode_pos if mode =='challenge' else practice_mode_pos
pag.click(x = start_pos[0], y = start_pos[1])
time.sleep(3)
err = 0

history = []
depth = 3
while True:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir, score = find_direct(tile_array, depth = depth)
    if err == 10: break
    if tile_array[3][0] == 0 and final_dir == 'left':
        time.sleep(0.1)
        tile_array = get_tile_array(tile_xpos, tile_ypos)
        final_dir, score = find_direct(tile_array, depth = depth)
        if final_dir == 'left':
            drag('left')
            time.sleep(0.15)
            drag('right')
            time.sleep(0.06)
            continue
    if score < 0:
        err += 1
        if err == 1:
            time.sleep(0.1)
            continue
        drag('up')
        time.sleep(0.15)
        drag('down')
        time.sleep(0.06)
        continue
    drag(final_dir)
    err = 0
    # history.append(tile_array)
    # history.append(final_dir)
    expect = np.array(after_moving(tile_array, final_dir))
    filled  = np.sum(expect != 0)
    depth = 3 + (filled//13)*2
    time.sleep((filled//13)*0.15+0.06)
    # depth = 3
    # time.sleep(0.1)

In [48]:
tile_array = get_tile_array(tile_xpos, tile_ypos)
tile_array
# find_direct(tile_array)  

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 0., 1.],
       [1., 1., 1., 1.]])

In [48]:
# history[0]
(np.sum(history[0] != 0)//8)*0.2

np.float64(0.0)

In [43]:
(5//8)*0.2

0.0

In [33]:
for ii in range(len(history)//2-1):
    pre = np.array(ohouse.after_moving(history[2*ii], history[2*ii+1]))
    aft = np.array(history[2*ii + 2])
    filled = pre != 0
    if (pre[filled] != aft[filled]).any():
        print(ii)
        print(pre)
        print(aft)

1
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [3. 0. 0. 0.]]
2
[[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]]
[[0. 1. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]]
8
[[3. 2. 2. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[3. 2. 2. 0.]
 [1. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
14
[[4. 2. 0. 0.]
 [3. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]]
[[4. 2. 1. 0.]
 [3. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]]
15
[[4. 2. 1. 0.]
 [3. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[4. 2. 1. 0.]
 [3. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 1. 0. 0.]]
18
[[4. 3. 0. 0.]
 [3. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 0.]]
[[4. 2. 0. 0.]
 [3. 0. 0. 0.]
 [2. 0. 0. 1.]
 [1. 0. 0. 0.]]
19
[[4. 2. 0. 1.]
 [3. 0. 0. 0.]
 [2. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[4. 3. 0. 1.]
 [3. 0. 0. 0.]
 [3. 0. 0. 0.]
 [0. 0. 1. 0.]]
20
[[4. 3. 1. 1.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[4. 3. 1. 1.]
 [3. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]
21


In [96]:
%%time
tile_array = np.array([[0, 0, 0, 0], 
                       [1, 0, 0, 0], 
                       [0, 0, 1, 0], 
                       [1, 0, 0, 0]])
results = ohouse.find_direction2(tile_array, depth=5)
results

CPU times: total: 15.6 ms
Wall time: 1.9 s


[('none', np.float64(10590360663.11732)),
 ('none', np.float64(10673990922.606812)),
 ('none', np.float64(9559799807.844862)),
 ('none', np.float64(9685873075.879747))]

In [95]:
%%time
tile_array = np.array([[0, 0, 0, 0], 
                       [1, 0, 0, 0], 
                       [0, 0, 1, 0], 
                       [1, 0, 0, 0]])
results = ohouse.find_direct(tile_array, depth=5)
# results

CPU times: total: 422 ms
Wall time: 717 ms


In [56]:
prepre = np.array([[9., 7., 6., 1.],
                    [0., 3., 6., 4.],
                    [0., 1., 2., 1.],
                    [0., 0., 1., 3.]])

pre = np.array(ohouse.after_moving(prepre, 'up'))
aft = np.array([[9., 7., 6., 1.],
                [0., 3., 2., 4.],
                [0., 1., 1., 1.],
                [0., 0., 0., 3.]])

filled = pre != 0
if (pre[filled] != aft[filled]).any():
    print(ii)
    print(pre)
    print(aft)

446
[[9. 7. 7. 1.]
 [0. 3. 2. 4.]
 [0. 1. 1. 1.]
 [0. 0. 0. 3.]]
[[9. 7. 6. 1.]
 [0. 3. 2. 4.]
 [0. 1. 1. 1.]
 [0. 0. 0. 3.]]


In [54]:
prepre

(array([[9., 7., 6., 1.],
        [0., 3., 6., 4.],
        [0., 1., 2., 1.],
        [0., 0., 1., 3.]]),)

In [ ]:
while True:
    tile_array = get_tile_array(tile_xpos, tile_ypos)
    final_dir, score = find_direct(tile_array, 0)
    print("\r", final_dir, score, end="")
    # time.sleep(0)

In [29]:
5 // 2

2

In [10]:
tile_array = get_tile_array(tile_xpos, tile_ypos)
find_direct(tile_array)   

('left', np.float64(-3.0))

In [58]:
aa = np.array([[0., 0., 2., 1.],
               [0., 0., 0., 0.],
               [0., 0., 0., 0.],
               [0., 0., 0., 0.]])
bb = np.array([[0., 0., 1., 1.],
               [0., 0., 0., 0.],
               [0., 0., 0., 1.],
               [0., 0., 0., 0.]])
filled = aa != 0
if (aa[filled] ==  bb[filled]).all(): 
    print('a')
else:
    print('b')

# np.logical_or(blank, new)

b


In [47]:
bb[filled]

array([1., 1.])

In [71]:
tile_array = np.array([[3, 2, 2, 4], 
                       [1, 2, 2, 0], 
                       [0, 0, 0, 0], 
                       [1, 1, 0, 0]])
# tile_array = get_tile_array(tile_xpos, tile_ypos)
final_dir, score = find_direct(tile_array)   
final_dir, score

('left', np.float64(-1.0))

In [68]:
pdb.pm()

> c:\users\khcho\miniforge3\lib\site-packages\multiprocess\pool.py(774)get()
    772             return self._value
    773         else:
--> 774             raise self._value
    775 
    776     def _set(self, i, obj):



ipdb>  u


> c:\users\khcho\appdata\local\temp\ipykernel_17728\2442304970.py(149)find_direction()



ipdb>  results


[<multiprocess.pool.ApplyResult object at 0x00000145452F1940>, <multiprocess.pool.ApplyResult object at 0x00000145449862D0>, <multiprocess.pool.ApplyResult object at 0x0000014544985A30>, <multiprocess.pool.ApplyResult object at 0x0000014544986270>]


...

ipdb>  results[0].get()


*** NameError: name 'np' is not defined


ipdb>  import numpy as np
ipdb>  results[0].get()


*** NameError: name 'np' is not defined


ipdb>  q


In [73]:
get_tile_array(tile_xpos, tile_ypos)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [67]:
tile_array == after_moving(tile_array, 'left')

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])